In [ ]:
#  
#  Author: Tanish Tyagi
#  

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

# machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix, average_precision_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn import preprocessing
import seaborn as sns
from sklearn.metrics import roc_auc_score, matthews_corrcoef, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# deep learning libraries
import torch
import transformers
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs

import time
import math
import random
from tqdm import tqdm
import regex as re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

## Loading in Patient Level Dataset with ClinicalBERT Sequence Level Predictions

In [ ]:
patient_level_preds = pd.read_csv(r"Storage/Bert/john_hsu_sequence_preds_proba.csv")

In [ ]:
patient_level_preds = patient_level_preds[patient_level_preds["syndromic_dx"].isna() == False]
patient_level_preds = patient_level_preds.reset_index(drop = True)

In [ ]:
patient_level_preds.columns

## Feature Engineering to Get Patient Level Labels

In [ ]:
for i in tqdm(range(len(patient_level_preds))):
    if (int(patient_level_preds.at[i, "syndromic_dx"]) > 0):
        patient_level_preds.at[i, "syndromic_dx"] = 1
    else:
        patient_level_preds.at[i, "syndromic_dx"] = 0

In [ ]:
patient_level_preds["syndromic_dx"].value_counts()

In [ ]:
patient_level_features = pd.DataFrame(columns = ["PatientID", "percent_yes", "percent_no", "percent_neither", "sequence_count", "label"])

## Percent yes, no, neither Feature Engineering

In [ ]:
data = []
for i in tqdm(range(len(patient_level_preds["PatientID"].unique()))):
    curr = patient_level_preds[patient_level_preds["PatientID"] == str(patient_level_preds["PatientID"].unique()[i])]
    seq_count = len(curr)

    if (seq_count <= 10):
        continue
    
    p_yes = len(curr[curr["class_pred"] == 2]) / len(curr)
    p_no = len(curr[curr["class_pred"] == 0]) / len(curr)
    p_ntr = len(curr[curr["class_pred"] == 1]) / len(curr)

    no_count = len(curr[curr["syndromic_dx"] == 0])
    yes_count = len(curr[curr["syndromic_dx"] == 1])

    label = 0
    if (yes_count > no_count):
        label = 1 
    
    curr_dict = {
        "PatientID" : str(patient_level_preds["PatientID"].unique()[i]),
        "percent_yes" : p_yes,
        "percent_no" : p_no,
        "percent_neither" : p_ntr,
        "sequence_count" : seq_count,
        "label" : label  
    }

    data.append(curr_dict)

patient_level_features = pd.DataFrame(data)

In [ ]:
x = patient_level_features["sequence_count"].describe()

## Converting Sequence Count Feature to a discrete value by bucketing based off quartiles

In [ ]:
for i in tqdm(range(len(patient_level_features))):
    if (patient_level_features.at[i, "sequence_count"] <= x["25%"]):
        patient_level_features.at[i, "sequence_count"] = 0
    elif (patient_level_features.at[i, "sequence_count"] <= x["50%"]):
        patient_level_features.at[i, "sequence_count"] = 1
    elif (patient_level_features.at[i, "sequence_count"] <= x["75%"]):
        patient_level_features.at[i, "sequence_count"] = 2
    else:
        patient_level_features.at[i, "sequence_count"] = 3

In [ ]:
# patient_level_features.to_csv(r"Storage/Bert/jh_patient_level_features.csv", index = False)

## Splitting into Train, Validation, Test Splits

In [ ]:
X = patient_level_features[["percent_yes", "percent_no", "percent_neither", "sequence_count"]]
y = patient_level_features["label"]

y_label = y.to_numpy()
X_train, X_test_valid, y_train, y_test_valid = train_test_split(X,y,test_size=0.15, stratify=y_label)

y_test_valid_label = y_test_valid.to_numpy()
X_valid, X_test, y_valid, y_test = train_test_split(X_test_valid, y_test_valid,test_size=0.5, stratify=y_test_valid_label)

### Feature Standardization

In [ ]:
num_cols = ["percent_yes","percent_no", "sequence_count"]

for i in num_cols:
    scale = StandardScaler().fit(X_train[[i]])

    X_train[i] = scale.transform(X_train[[i]])
    X_valid[i] = scale.transform(X_valid[[i]])
    X_test[i] = scale.transform(X_test[[i]]) 

In [ ]:
len(X_train), len(X_valid), len(X_test)

In [ ]:
X_cross_validation = pd.concat([X_train, X_valid]).to_numpy()
y_cross_validation = pd.concat([y_train, y_valid]).to_numpy()

## K-Fold Cross Validation Training Loop

In [ ]:
kf = KFold(n_splits = 10, shuffle = True)
kf.get_n_splits(X_cross_validation)

In [ ]:
def logisitic_regression(X_train, y_train, X_test, y_test, c, want_conf_mat):
    # fitting model
    lr = LogisticRegression(penalty = 'l1', solver = 'liblinear', C = c, random_state = 0, class_weight = 'balanced')
    lr.fit(X_train, y_train)
    
    # predictions
    y_pred = lr.predict(X_test)
    y_prob = lr.predict_proba(X_test)

    # collecting results
    acc = metrics.accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob[:, 1])
    
    # if (save_model == True):
    #     pickle.dump(lr, open("Storage/Model/" + name, 'wb'))

    
    if (want_conf_mat == True):
        return lr, acc, auc, c, confusion_matrix(y_test, y_pred)
        
    return lr, acc, auc, c

In [ ]:
counter = 0
df_list  = []

for train_index, test_index in kf.split(X_cross_validation):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
    y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]

    #X_train_cv = X_train_cv.drop(columns = ["PatientID"])
    #X_test_cv = X_test_cv.drop(columns = ["PatientID"])

    acc_list = []
    auc_list = []
    c_list = []

    # tuning for optimal lambda value
    for c in [0.01, 0.1, 1, 10, 100]:
        #name = "Fold-" + str((counter + 1)) + "-Corr-" + str(corr) + "-C-" + str(c) + ".sav"
        lr, acc, auc, c = logisitic_regression(X_train_cv, y_train_cv, X_test_cv, y_test_cv, c, False)
        acc_list.append(acc)
        auc_list.append(auc)
        c_list.append(c)
    
    # gathering model stats
    acc_df = pd.DataFrame(acc_list, columns=['acc'])
    auc_df = pd.DataFrame(auc_list, columns=['auc'])
    c_df = pd.DataFrame(c_list, columns=['c_value'])
    
    assert len(acc_df) == len(auc_df) == len(c_df)
        
    iter_df = pd.concat([c_df, acc_df, auc_df], axis=1)
    iter_df['fold_number'] = [(counter + 1)] * len(iter_df)
    df_list.append(iter_df)
        
    print("Completed Fold #: ", counter + 1)
    counter += 1
    
    print("Stats DF has", len(df_list), "records")

In [ ]:
all_df = pd.concat(df_list)

## Finding Optimal Hyperparameters

In [ ]:
average_results_df = []

for c in [0.01, 0.1, 1, 10, 100]:
    filtered = all_df[(all_df["c_value"] == c)]
    avg_auc = filtered["auc"].mean()
    avg_acc = filtered["acc"].mean()

    filler = np.arange(5, 8)**2
    df = pd.DataFrame(filler.reshape(1, 3), columns = ["c_value", "acc", "auc"])
    df.loc[df.index] = [c, avg_acc, avg_auc]
    #print(df)
    
    average_results_df.append(df)

In [ ]:
average_results_df = pd.concat(average_results_df)

In [ ]:
average_results_df[average_results_df['auc'] == max(average_results_df['auc'])]

## Model Evaluation on Held Out Test Set

In [ ]:
c = 100

lr = LogisticRegression(penalty = 'l1', solver = 'liblinear', C = c, class_weight = 'balanced')
lr.fit(X_train, y_train)

In [ ]:
y_test_preds = lr.predict(X_test)

## Accuracy and AUC

In [ ]:
acc = metrics.accuracy_score(y_test, y_test_preds)
acc

In [ ]:
y_test_prob = lr.predict_proba(X_test)
auc = roc_auc_score(y_test, y_test_prob[:, 1])
auc

## Classification Report

In [ ]:
target_names = ['Negative', 'Positive']
results_lgr = classification_report(y_test, y_test_preds, target_names = target_names, output_dict=True)
results_lgr = pd.DataFrame(results_lgr).transpose()
results_lgr

## Saving Model and features plus ROC Curves

In [ ]:
save = False

if save == True:
    file_name = "lr_12_26_patient_level.sav"
    pickle.dump(lr, open(file_name, 'wb'))

    X_train_df = pd.concat([X_train, y_train])
    X_valid_df = pd.concat([X_valid, y_valid])
    X_test_df = pd.concat([X_test, y_test])

    X_train_df.to_csv(r"Storage/Bert/patient_level_train_85.csv", index = False)
    X_valid_df.to_csv(r"Storage/Bert/patient_level_valid_85.csv", index = False)
    X_test_df.to_csv(r"Storage/Bert/patient_level_test_85.csv", index = False)
    n_class = 2

    fpr = {}
    tpr = {}
    thresh = {}
    roc_auc = {}

    for i in range(n_class):    
        fpr[i], tpr[i], thresh[i] = metrics.roc_curve(y_test, y_test_prob[:,i], pos_label = i)
        roc_auc[i] = metrics.auc(fpr[i], tpr[i])
        
    lw = 2

    # plotting    
    plt.plot(fpr[0], tpr[0], linestyle='--', color='red', label='L1 Logistic Regression (area = %0.2f)' % roc_auc[0])
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive rate')
    plt.legend(loc='best')

    plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='L1 Logistic Regression (area = %0.2f)' % roc_auc[1])
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive rate')
    plt.legend(loc='best')
    plt.show()

    # plt.savefig("Storage/Bert/patient_level_roc_85.svg")


## Further Metrics

In [ ]:
conf_mat = confusion_matrix(y_test, y_test_preds)

In [ ]:
import numpy as np
FP = conf_mat.sum(axis = 0) - np.diag(conf_mat) 
FN = conf_mat.sum(axis = 1) - np.diag(conf_mat)
TP = np.diag(conf_mat)
TN = conf_mat.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)

# Specificity or true negative rate
TNR = TN/(TN+FP) 

# Precision or positive predictive value
PPV = TP/(TP+FP)

# Negative predictive value
NPV = TN/(TN+FN)

# Fall out or false positive rate
FPR = FP/(FP+TN)

# False negative rate
FNR = FN/(TP+FN)

# False discovery rate
FDR = FP/(TP+FP)

print("Sensitivity: ", TPR)
print("Specificity: ", TNR)
print("NPV: ", NPV)
print("PPV: ", PPV)
print("FPR: ", FPR)